In [37]:
import nltk
# nltk.download('stopwords')

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

[nltk_data] Downloading package stopwords to C:\Users\Kshitiz
[nltk_data]     sharan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [38]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [39]:
data_file='./train1.csv'
data=pd.read_csv(data_file)

data.head()

,score,Text
0,5,dr. goldberg offers everything i look for in a...
1,2,"Unfortunately, the frustration of being Dr. Go..."
2,4,Been going to Dr. Goldberg for over 10 years. ...
3,4,Got a letter in the mail last week that said D...
4,1,I don't know what Dr. Goldberg was like before...


In [40]:
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

In [41]:
actualScore_train=data['score']
positiveNegative=actualScore_train.map(partition)
data['score']=positiveNegative

In [42]:
data['gt'] = data['score'].map({'negative':0,'positive':1})

data['Text']=data['Text'].map(preprocess_sentence)

In [55]:
num_classes=len(data.Text.unique())
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [56]:
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [57]:
max_len=32
sentences=data['Text']
labels=data['gt']
len(sentences),len(labels)

(650000, 650000)

In [60]:
dbert_tokenizer.tokenize(sentences[0])

['goldberg',
 'offers',
 'everything',
 'look',
 'general',
 'practitioner',
 'nice',
 'easy',
 'talk',
 'without',
 'patron',
 '##izing',
 'always',
 'time',
 'seeing',
 'patients',
 'affiliated',
 'top',
 'notch',
 'hospital',
 'nyu',
 'parents',
 'explained',
 'important',
 'case',
 'something',
 'happens',
 'need',
 'surgery',
 'get',
 'refer',
 '##ral',
 '##s',
 'see',
 'specialists',
 'without',
 'see',
 'first',
 'really',
 'need',
 'sitting',
 'trying',
 'think',
 'complaints',
 'really',
 'drawing',
 'blank']

In [62]:
dbert_inp=dbert_tokenizer.encode_plus(sentences[0],add_special_tokens = True,max_length =20,pad_to_max_length = True,truncation=True)
dbert_inp

{'input_ids': [101, 18522, 4107, 2673, 2298, 2236, 18742, 3835, 3733, 2831, 2302, 9161, 6026, 2467, 2051, 3773, 5022, 6989, 2327, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [63]:
dbert_inp['input_ids']

[101,
 18522,
 4107,
 2673,
 2298,
 2236,
 18742,
 3835,
 3733,
 2831,
 2302,
 9161,
 6026,
 2467,
 2051,
 3773,
 5022,
 6989,
 2327,
 102]

In [64]:
id_inp=np.asarray(dbert_inp['input_ids'])
mask_inp=np.asarray(dbert_inp['attention_mask'])
out=dbert_model([id_inp.reshape(1,-1),mask_inp.reshape(1,-1)])
type(out),out

(transformers.modeling_tf_outputs.TFBaseModelOutput,
 TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 20, 768), dtype=float32, numpy=
 array([[[-1.98320448e-01,  3.00513357e-02,  1.59358427e-01, ...,
          -1.69523388e-01,  4.42057252e-01,  1.07091025e-01],
         [ 2.34363168e-01,  4.54131216e-02,  9.03326496e-02, ...,
           5.73578440e-02,  1.72908157e-01,  2.74978667e-01],
         [-3.47497255e-01,  2.88471729e-01,  3.91802549e-01, ...,
          -2.27810785e-01,  2.29090795e-01, -1.70055494e-01],
         ...,
         [ 2.42646620e-01,  1.87672615e-01,  3.72264087e-01, ...,
           1.08914450e-04,  1.46245599e-01, -1.59351721e-01],
         [-3.69360335e-02,  6.71055168e-02,  8.68291110e-02, ...,
           1.13608122e-01,  3.42743218e-01, -4.49185669e-01],
         [ 7.01780379e-01,  2.64715254e-01, -3.71728539e-01, ...,
           7.89778829e-02, -4.27991748e-01, -2.50801206e-01]]],
       dtype=float32)>, hidden_states=None, attentions=None))

In [65]:
out[0][:,0,:]

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.98320448e-01,  3.00513357e-02,  1.59358427e-01,
        -1.54239729e-01,  4.71313633e-02, -8.48967433e-02,
         3.48035991e-01,  3.43655705e-01, -7.00134486e-02,
        -3.34983110e-01,  1.90282896e-01, -1.33307323e-01,
         2.04382807e-01,  2.77472168e-01, -1.21448442e-01,
         2.54669219e-01,  1.14152081e-01,  2.88363129e-01,
         1.48429453e-01, -1.32481694e-01,  4.83718403e-02,
        -6.26427710e-01,  2.55077124e-01, -1.39423832e-01,
        -5.51867485e-03,  8.52848589e-03, -4.97719944e-02,
        -2.30354980e-01, -4.35163788e-02,  1.69881523e-01,
         1.70469880e-01,  1.27865836e-01, -8.48859549e-03,
        -3.59298974e-01,  1.43056065e-01, -9.18514654e-02,
         3.07672590e-01,  2.68631309e-01, -7.78873563e-02,
         1.63533799e-02, -1.18057154e-01,  1.11377575e-02,
         3.44903141e-01, -1.88439608e-01,  5.97983412e-02,
        -2.17247829e-01, -2.53238463e+00, -3.55465353e-01,
      

In [66]:
dbert_tokenizer.decode(dbert_inp['input_ids'])

'[CLS] goldberg offers everything look general practitioner nice easy talk without patronizing always time seeing patients affiliated top [SEP]'

In [67]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model

In [68]:
model=create_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_2 (TFDisti TFBaseModelOutput(la 66362880    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 768)          0           tf_distil_bert_model_2[0][0] 

In [69]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [70]:
len(input_ids),len(attention_masks),len(labels)

(650000, 650000, 650000)

In [73]:
print('Preparing the pickle file.....')

pickle_inp_path='./dbert_inp.pkl'
pickle_mask_path='./dbert_mask.pkl'
pickle_label_path='./dbert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Preparing the pickle file.....
Pickle files saved as  ./dbert_inp.pkl ./dbert_mask.pkl ./dbert_label.pkl


In [74]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (650000, 32) Attention mask shape (650000, 32) Input label shape (650000,)


In [75]:
label_class_dict={0:'ham',1:'spam'}
target_names=label_class_dict.values()

In [76]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))


log_dir='dbert_model'
model_save_path='./dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

Train inp shape (520000, 32) Val input shape (130000, 32)
Train label shape (520000,) Val label shape (130000,)
Train attention mask shape (520000, 32) Val attention mask shape (130000, 32)


In [77]:
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

In [ ]:
# TRAINING (It is taking alot of time on my computer)

history=model.fit([train_inp,train_mask],train_label,batch_size=16,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)